# Chat Room of Models

What if we put different models in the same chat room? How would they respond to each other? 

blog link : https://jumbled-marmoset-78e.notion.site/reading-and-writing-electronic-text-week10-zoecai-9d2c9810a8884f1085a7bd8a0fc2d251

## 0 hide pink blocks

In [143]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

## 1 Markov Chain - Markovify - just a review:)

In [1]:
textMC = open("WALDEN.txt").read()
print(textMC[:200])

WALDEN
Economy
When I wrote the following pages, or rather the bulk of them, I lived alone, in the woods, a mile from any neighbor, in a house which I had built myself, on the shore of Walden Pond, in


In [2]:
import random
wordMC = textMC.split()
# random.sample(textMC,10)
random.sample(wordMC,10)

['and',
 'the',
 'them',
 'there',
 'definite,',
 'party',
 'London,',
 'I',
 'to',
 'unmusical']

In [3]:
from collections import Counter
# Counter(textMC).most_common(12)
Counter(wordMC).most_common(12)

[('the', 6909),
 ('and', 4434),
 ('of', 3461),
 ('to', 3025),
 ('a', 2940),
 ('I', 1967),
 ('in', 1930),
 ('is', 1246),
 ('that', 1245),
 ('as', 1132),
 ('it', 1086),
 ('not', 996)]

In [4]:
import sys
!{sys.executable} -m pip install markovify

In [5]:
import markovify

In [6]:
generatorMC = markovify.Text(textMC)
print(generatorMC.make_sentence())

What avails it that men are satisfied if they were just as large to me because he belongs to himself and his father’s or his nature does not, that he should live on hard, raw corn on the main object were to raise the echoes by striking with a peculiar sense a part of his head.


In [7]:
print(generatorMC.make_short_sentence(50,tries=100))

When they make their port.


## 1 Markov Chain - N-gram

In [8]:
textMC = open("WALDEN.txt").read()
wordMC = textMC.split()
pairMC = []
for i in range(len(wordMC)-5):
    pair = (wordMC[i], wordMC[i+1], wordMC[i+2], wordMC[i+3], wordMC[i+4], wordMC[i+5])
    pairMC.append(pair)
print(pairMC[:10])

[('WALDEN', 'Economy', 'When', 'I', 'wrote', 'the'), ('Economy', 'When', 'I', 'wrote', 'the', 'following'), ('When', 'I', 'wrote', 'the', 'following', 'pages,'), ('I', 'wrote', 'the', 'following', 'pages,', 'or'), ('wrote', 'the', 'following', 'pages,', 'or', 'rather'), ('the', 'following', 'pages,', 'or', 'rather', 'the'), ('following', 'pages,', 'or', 'rather', 'the', 'bulk'), ('pages,', 'or', 'rather', 'the', 'bulk', 'of'), ('or', 'rather', 'the', 'bulk', 'of', 'them,'), ('rather', 'the', 'bulk', 'of', 'them,', 'I')]


### 1.1 n-gram - way1

In [9]:
from collections import Counter
countMC = Counter(pairMC)
countMC.most_common(10)

[(('two', 'acres', 'and', 'a', 'half', 'of'), 3),
 (('I', 'am', 'no', 'more', 'lonely', 'than'), 3),
 (('It', 'is', 'not', 'worth', 'the', 'while'), 3),
 (('is', 'not', 'worth', 'the', 'while', 'to'), 3),
 (('to', 'the', 'other', 'side', 'of', 'the'), 2),
 (('to', 'hear', 'what', 'was', 'in', 'the'), 2),
 (('I', 'planted', 'about', 'two', 'acres', 'and'), 2),
 (('planted', 'about', 'two', 'acres', 'and', 'a'), 2),
 (('about', 'two', 'acres', 'and', 'a', 'half'), 2),
 (('of', 'which', 'I', 'have', 'not', 'rendered'), 2)]

### 1.2 n-gram - way2

In [10]:
import random
def ngrams_for_sequence(n, seq):
    return [tuple(seq[i:i+n]) for i in range(len(seq)-n+1)]

gram6 = ngrams_for_sequence(6, open("WALDEN.txt").read().split())
# random.sample(gram6, 10)
Counter(gram6).most_common(10)

[(('two', 'acres', 'and', 'a', 'half', 'of'), 3),
 (('I', 'am', 'no', 'more', 'lonely', 'than'), 3),
 (('It', 'is', 'not', 'worth', 'the', 'while'), 3),
 (('is', 'not', 'worth', 'the', 'while', 'to'), 3),
 (('to', 'the', 'other', 'side', 'of', 'the'), 2),
 (('to', 'hear', 'what', 'was', 'in', 'the'), 2),
 (('I', 'planted', 'about', 'two', 'acres', 'and'), 2),
 (('planted', 'about', 'two', 'acres', 'and', 'a'), 2),
 (('about', 'two', 'acres', 'and', 'a', 'half'), 2),
 (('of', 'which', 'I', 'have', 'not', 'rendered'), 2)]

### 1.3 n-gram - way3

In [11]:
def add_to_model(model, n, seq):
    # make a copy of seq and append None to the end
    seq = list(seq[:]) + [None]
    for i in range(len(seq)-n):
        # tuple because we're using it as a dict key!
        gram = tuple(seq[i:i+n])
        next_item = seq[i+n]            
        if gram not in model:
            model[gram] = []
        model[gram].append(next_item)

def markov_model(n, seq):
    model = {}
    add_to_model(model, n, seq)
    return model

In [12]:
genesis_markov_model = markov_model(1, open("WALDEN.txt").read().split())
genesis_markov_model

{('WALDEN',): ['Economy'],
 ('Economy',): ['When', 'is'],
 ('When',): ['I',
  'we',
  'one',
  'a',
  'he',
  'Madam',
  'I',
  'I',
  'the',
  'I',
  'I',
  'he',
  'I',
  'I',
  'I',
  'it',
  'you',
  'men',
  'the',
  'you',
  'I',
  'a',
  'formerly',
  'Phaeton,',
  'first',
  'I',
  'we',
  'the',
  'my',
  'I',
  'the',
  'other',
  'I',
  'the',
  'visitors',
  'Winslow,',
  'the',
  'I',
  'Nature',
  'I',
  'I',
  'I',
  'my',
  'I',
  'there',
  'they',
  'only',
  'the',
  'you',
  'the',
  'I',
  'I',
  'I',
  'the',
  'they',
  'I',
  'I',
  'I',
  'compelled',
  'chestnuts',
  'I',
  'I',
  'the',
  'the',
  'the',
  'I',
  'the',
  'I',
  'the',
  'I',
  'I',
  'I',
  'this',
  'the',
  'two',
  'I',
  'such',
  'a',
  'the',
  'the',
  'the',
  'I',
  'the',
  'the',
  'the',
  'the',
  'the',
  'I',
  'I',
  'the',
  'they',
  'I',
  'I',
  'many'],
 ('I',): ['wrote',
  'lived',
  'had',
  'lived',
  'am',
  'should',
  'got',
  'did',
  'was',
  'devoted',
  'mainta

In [13]:
# genesis_markov_model[('I',)]
Counter(genesis_markov_model).most_common(10)

[(('sang,—',), ['“we']),
 (('shoulders',), ['“until']),
 (('surprise,',), ['“to', 'that', 'that', 'that']),
 (('bream;',), ['“these']),
 (('Evelyn,',), ['“the']),
 (('deemed',), ['“the']),
 (('macularius)',), ['“teter”']),
 (('he;',), ['“some', 'I']),
 (('Thseng-tseu,',), ['“one']),
 (('Philadelphia',), ['“nearly'])]

### 2.1 generate markov chain sentence according to the input

In [14]:
import random
def gen_from_model(n, model, start=None, max_gen=100):
    if start is None:
        start = random.choice(list(model.keys()))
    output = list(start)
    for i in range(max_gen):
        start = tuple(output[-n:])
        next_item = random.choice(model[start])
        if next_item is None:
            break
        else:
            output.append(next_item)
    return output

In [15]:
genesis_word_model = markov_model(1, open("WALDEN.txt").read().split())
type(genesis_word_model)

dict

In [57]:
genesis_word_model = markov_model(1, open("WALDEN.txt").read().split())
input_sentence = 'I feel good. What about you?'
input_word = input_sentence.split()
input_word = tuple(input_word)
total_word = 140
generated_words = gen_from_model(1, genesis_word_model, input_word, total_word)
# print(' '.join(generated_words))

sentences = []
sentence_words = [] 
for word in generated_words:
    sentence_words.append(word)
    if word.endswith("." or "?" or "!"):
        sentences.append(" ".join(sentence_words))
        sentence_words = []

print(sentences)
sentences[2]

['you? Are its length, in Louisiana bayous.', 'When the warmest weather I did not only thoroughly cured for our country affords, purely brute or else in my shell, and a great gold and now found wanting.', 'A robust but leave very warm, but a foreign as the vert more awful to rumble with their hands of the Niger, or important distinction between the country without any case.', 'Referred to pull so sweet gum would not made of England, says that way, and beholding the horizon, I sat, even then unnecessary; the wine and paint a hunter’s cottage window, and all that.']


'A robust but leave very warm, but a foreign as the vert more awful to rumble with their hands of the Niger, or important distinction between the country without any case.'

In [17]:
dialogue1 = sentences[1]

## 2 RNN

In [18]:
# pip install tensorflow

In [19]:
# from tensorflow.keras.callbacks import LambdaCallback
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras.utils import get_file
# import numpy as np
# import random
# import sys
# import io
# import requests
# import re

In [20]:
# r = requests.get("https://data.heatonresearch.com/data/t81-558/text/"\
#                  "treasure_island.txt")
# raw_text = r.text
# type(raw_text)
# # print(raw_text[0:1000])

In [21]:
# raw_text = open("WALDEN.txt").read()

In [22]:
# processed_text = raw_text.lower()
# processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text) 

# print('corpus length:', len(processed_text))

# chars = sorted(list(set(processed_text)))
# print('total chars:', len(chars))
# char_indices = dict((c, i) for i, c in enumerate(chars))
# indices_char = dict((i, c) for i, c in enumerate(chars))

In [23]:
# # cut the text in semi-redundant sequences of maxlen characters
# maxlen = 40
# step = 3
# sentences = []
# next_chars = []
# for i in range(0, len(processed_text) - maxlen, step):
#     sentences.append(processed_text[i: i + maxlen])
#     next_chars.append(processed_text[i + maxlen])
# print('nb sequences:', len(sentences))

In [24]:
# sentences

In [25]:
# #convert the text into vectors
# print('Vectorization...')
# x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
# y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
# for i, sentence in enumerate(sentences):
#     for t, char in enumerate(sentence):
#         x[i, t, char_indices[char]] = 1
#     y[i, char_indices[next_chars[i]]] = 1

In [26]:
# #create the neural network. This neural network's primary feature is the LSTM layer, which allows the sequences to be processed
# # build the model: a single LSTM
# print('Build model...')
# model = Sequential()
# model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model.add(Dense(len(chars), activation='softmax'))

# optimizer = RMSprop(lr=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [27]:
# model.summary()

In [28]:
# def sample(preds, temperature=1.0):
#     # helper function to sample an index from a probability array
#     preds = np.asarray(preds).astype('float64')
#     preds = np.log(preds) / temperature
#     exp_preds = np.exp(preds)
#     preds = exp_preds / np.sum(exp_preds)
#     probas = np.random.multinomial(1, preds, 1)
#     return np.argmax(probas)

In [29]:
# def on_epoch_end(epoch, _):
#     # Function invoked at end of each epoch. Prints generated text.
#     print("******************************************************")
#     print('----- Generating text after Epoch: %d' % epoch)

#     start_index = random.randint(0, len(processed_text) - maxlen - 1)
#     for temperature in [0.2, 0.5, 1.0, 1.2]:
#         print('----- temperature:', temperature)

#         generated = ''
#         sentence = processed_text[start_index: start_index + maxlen]
#         generated += sentence
#         print('----- Generating with seed: "' + sentence + '"')
#         sys.stdout.write(generated)

#         for i in range(10):
#             x_pred = np.zeros((1, maxlen, len(chars)))
#             for t, char in enumerate(sentence):
#                 x_pred[0, t, char_indices[char]] = 1.

#             preds = model.predict(x_pred, verbose=0)[0]
#             next_index = sample(preds, temperature)
#             next_char = indices_char[next_index]

#             generated += next_char
#             sentence = sentence[1:] + next_char

#             sys.stdout.write(next_char)
#             sys.stdout.flush()
#         print()

In [30]:
# # Ignore useless W0819 warnings generated by TensorFlow 2.0.  Hopefully can remove this ignore in the future.
# # See https://github.com/tensorflow/tensorflow/issues/31308
# import logging, os
# logging.disable(logging.WARNING)
# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# # Fit the model
# # print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# model.fit(x, y,
#           batch_size=128,
#           epochs=60,
#           callbacks=[print_callback])


## 3 GPT-2

In [31]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [32]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [33]:
inputs = tokenizer.encode(dialogue1, return_tensors='pt')

In [34]:
outputs = model.generate(inputs,max_length=100, do_sample=True,temperature=1.0, top_k = 1000)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [35]:
dialogue3 = tokenizer.decode(outputs[0],skip_special_tokens=True)
print(dialogue3)
# type(dialogue3)

What about you? It is as to say, buried themselves. They have been knocked down through a door or door-door. And now they are in thick smoke, down by doors, and all that. They are brought back to their houses by bus, by bus. All the traffic has apparently stopped and they have no story. They have no news. Now they look for someone. They search there for it. They keep searching, for the guy who brought them up. And they find him


In [36]:
GPT_words = dialogue3.split()
type(GPT_words)

GPT_sentences=[]
GPT_sentence_words=[]
for word in GPT_words:
    GPT_sentence_words.append(word)
    if word.endswith("." or "?" or "!"):
        GPT_sentences.append(" ". join(GPT_sentence_words))
        GPT_sentence_words=[]
        
print(GPT_sentences)
GPT_sentences[1]

['What about you? It is as to say, buried themselves.', 'They have been knocked down through a door or door-door.', 'And now they are in thick smoke, down by doors, and all that.', 'They are brought back to their houses by bus, by bus.', 'All the traffic has apparently stopped and they have no story.', 'They have no news.', 'Now they look for someone.', 'They search there for it.', 'They keep searching, for the guy who brought them up.']


'They have been knocked down through a door or door-door.'

In [37]:
dialogue3 = GPT_sentences[1]

## 4 Combination(without RNN)

In [144]:
markov_words = open("WALDEN.txt").read().split()

def to_sentence(inp):
    if isinstance(inp, list):
        string = " ".join(inp)
    else:
        string = inp
    string = string.replace("\t", " ")
    string = string.replace("\n", " ")
    sentences = [s.strip() for s in string.split('.') if len(s) != 0 and s != " "]
    return sentences

def markov(sentence):
    sentence = sentence.strip()
    if not sentence.endswith("."):
        sentence += '.'
    if sentence.split(" ")[-1] not in markov_words:
        return []

    generated_words = gen_from_model(1, genesis_word_model, tuple(sentence.split()), 60)
#     print("+", " ".join(generated_words))
    reply = to_sentence(generated_words)
    if len(reply) <= 1:
        return []
    else:
        return reply[1]
    
    
def gpt(sentence):
    sentence = sentence.strip()
    if not sentence.endswith("."):
        sentence += '.'
    inputs = tokenizer.encode(sentence, return_tensors='pt')
    outputs = model.generate(inputs, max_length=60, do_sample=True, temperature=1.0, top_k = 1000)
    content = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print("-", content)
    reply = to_sentence(content)
    if len(reply)<=1:
        return []
    return reply[1]

In [152]:
# import time
import random
from datetime import date, datetime

day = date.today().strftime('%b-%d-%Y')
                            
print("Hi cybersurfer, what's your name?\n")
name = input("You: ")
print()
# time.sleep(1)
print("Hi",name,"! What topic are you interested in?\n")
content = input("You: ")
print("\n")
# time.sleep(3)
print("—————— Welcome",name,"to the chat room #19834! —————\n")
# time.sleep(5)

while True:
    speaker = ''
    if random.random() > 0:
        reply = markov(content)
        speaker = "Markov"
    else:
        reply = gpt(content)
        speaker = 'GPT-2'
    if reply == []:
        reply = gpt(content)
        speaker = 'GPT-2'
    if reply == []:
        print("...")
        break
    curr_time = f"{day} {datetime.now().strftime('%H:%M:%S')}"
    print(f"{curr_time} \033[1;7m{speaker}\033[0m : {reply}")
    content = reply
#     time.sleep(5)
#     input("here")
    
# print(gpt("happy new year"))
# print(markov("happy new year"))


Hi cybersurfer, what's your name?

You: Joe

Hi Joe ! What topic are you interested in?

You: I think so I am


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




—————— Welcome Joe to the chat room #19834! —————

Apr-18-2022 05:47:42 Markov : I caught my best society, but I will not hear me of a sick man, I learned to conceive of no fox rolling on your own view with his hounds, and rains and force has faith, he must be fed me as a regular intervals scattered over the Governor or from storm as you can make this way to time
Apr-18-2022 05:47:42 Markov : Yet he has been killed in an inch thick; and above my sight of the Golden Age


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Apr-18-2022 05:47:44 GPT-2 : Hurrient praises Borja, and bestows upon his chosen deity a shining wondrous crown


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Apr-18-2022 05:48:17 GPT-2 : 'Oh dear dear!' said he, 'let him get out of his shadow and look at the dancing monkeys'


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Apr-18-2022 05:48:19 GPT-2 : As Heinemann ran his music box into a van, he met his first match with Corderidou


KeyboardInterrupt: 

In [126]:
a = 'asdf'
f"{a} qwer"
a + " qwer"

'asdf qwer'